In [ ]:
#Imports
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as mtick
import numpy as np
from matplotlib.ticker import MultipleLocator
import MDAnalysis as mda
from MDAnalysis.analysis import contacts
from MDAnalysis.analysis import distances
import pandas as pd
%matplotlib inline
import os
#from scikits.bootstrap import ci
from ci import ci_time_correl
from uncertainties import ufloat

In [ ]:
#Treating data
compounds = ["ADOS", "ADP", "ALO", "AMP", "ATP", "DMNFQ", "DMNFQH2", "FAD", "FMN", "GBZ", "HAQO", "HAQOH", "HQNO", "HQNOH", "ISO", "MND", "MNDOL", "MNQ", "MNQOL", "NAD+", "NADH", "NADP+", "NADPH", "NRBOS", "PQ", "PQOL", "RBFL", "RIB", "THI", "TMP", "TPP", "UBQ0", "UBQ0H2", "UBQN", "UBQNOL", "XQ", "XQH2"]
simulations = ["AA", "CG"]

SASAsAAvaluesdict = {}
SASAsCGvaluesdict = {}
SASAsAACIS = {}
SASAsCGCIS = {}
MeansAA = {}
MeansCG = {}
path_absolute = os.path.abspath(".")
for compound in compounds:
    path = path_absolute + f"/{compound}"
    if not os.path.isdir(path):
        continue
    os.chdir(path)
    SASAsAAvalues = np.loadtxt("SASA_AA.xvg", comments=("#","@")) if os.path.isfile("SASA_AA.xvg") else None
    if SASAsAAvalues is not None and len(SASAsAAvalues.shape) != 1:
        SASAsAAvalues = SASAsAAvalues[:,1]
        SASAsAACIS[compound] = ci_time_correl(SASAsAAvalues)
        MeansAA[compound] = (SASAsAACIS[compound][1] + SASAsAACIS[compound][0])/2
    else:
        SASAsAAvalues = SASAsAAvalues[1]
        SASAsAACIS[compound] = "None"
        MeansAA[compound] = SASAsAAvalues
    SASAsAAvaluesdict[compound] = SASAsAAvalues
    SASAsCGvalues = np.loadtxt("SASA_CG.xvg", comments=("#","@")) if os.path.isfile("SASA_CG.xvg") else None
    if SASAsCGvalues is not None and len(SASAsCGvalues.shape) != 1:
        SASAsCGvalues = SASAsCGvalues[:,1]        
        SASAsCGCIS[compound] = ci_time_correl(SASAsCGvalues)
        MeansCG[compound] = (SASAsCGCIS[compound][1] + SASAsCGCIS[compound][0])/2
    else:
        SASAsCGvalues = SASAsCGvalues[1]
        SASAsCGCIS[compound] = "None"
        MeansCG[compound] = SASAsCGvalues
    SASAsCGvaluesdict[compound] = SASAsCGvalues
    
SASAsAAdifferences = {}
SASAsCGdifferences = {}

for compound, vals in SASAsAACIS.items():
    if type(vals) != type(str()):
        minimum, maximum = [abs(val - MeansAA[compound]) for val in vals]# vals != 'None']
        SASAsAAdifferences[compound] = [minimum, maximum]
    else:
        SASAsAAdifferences[compound] = [0, 0]

for compound, vals in SASAsCGCIS.items():
    if type(vals) != type(str()):
        minimum, maximum = [abs(val - MeansCG[compound]) for val in vals]# vals != 'None']
        SASAsCGdifferences[compound] = [minimum, maximum]
    else:
        SASAsCGdifferences[compound] = [0, 0]

In [ ]:
#Individual plots
colors = ["#4bacc6","#f39517"]
pos = np.arange(1,2,.5)
labels = ['AA', 'CG']
ft_labels = 16

for compound in compounds:
    aa_reference = float("{:.3f}".format(MeansAA[compound]))
    aa_reference_std = float("{:.3f}".format(SASAsAAdifferences[compound][0]))

    cg_reference = float("{:.3f}".format(MeansCG[compound]))
    cg_reference_std = float("{:.3f}".format(SASAsCGdifferences[compound][0]))

    heights = [aa_reference, cg_reference]
    yerrs = np.array((aa_reference_std, cg_reference_std))
    if type(SASAsAACIS[compound]) != type(str()) or type(SASAsCGCIS[compound]) != type(str()):
        fig, ax = plt.subplots()
        ax.bar(pos, heights, yerr=yerrs, tick_label=labels, width=.3, linewidth=2, color=colors, capsize=5, error_kw={'lw':2}, edgecolor='black')
        ax.set_ylim(0, 1.5*max(aa_reference, cg_reference))
        ax_ticks = np.arange(0, max(ax.get_yticks()), 2)
        ax.set_yticks(ax_ticks)
        ax.axes.xaxis.set_ticklabels(labels, fontsize=ft_labels, weight='bold')
        ax.set_yticklabels(ax_ticks, fontsize=ft_labels, weight='bold')
        ax.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:.0f}'))
        ax.xaxis.tick_bottom()
        ax.tick_params(labeltop=False)  # don't put tick labels at the top
        ax.text(1, aa_reference+aa_reference_std+1, f'{aa_reference:.3f} ± {aa_reference_std:.3f}', ha='center', fontsize=ft_labels, color='#595959', weight='bold')
        ax.text(1.5, cg_reference+cg_reference_std+1, f'{cg_reference:.3f} ± {cg_reference_std:.3f}', ha='center', fontsize=ft_labels, color='#595959', weight='bold')

        d = .3  # proportion of vertical to horizontal extent of the slanted line

        for axis in ['top','bottom','left','right']:
            ax.spines[axis].set_linewidth(2)
        ax.tick_params(width=2, length=4)


        if compound == "DMNFQH2":
            compound = "DMNFQH$_2$"
        if compound == "UBQH2":
            compound = "UBQH$_2$"
        if compound == "UBQ0":
            compound = "UBQ-0"
        if compound == "NAD+":
            compound = "NAD$^+$"
        if compound == "NADP+":
            compound = "NADP$^+$"
        if compound == "UBQ0H2":
            compound = "UBQ0H$_2$"
        if compound == "XQH2":
            compound = "XQH$_2$"
        ax.set_title(f"{compound}", verticalalignment='center', fontsize=22, weight='bold', pad= "10.0")
        ax.set_ylabel("SASA (nm$^{2}$)", fontsize=ft_labels, weight='bold')
        figure = plt.gcf() # get current figure
        figure.set_size_inches(8, 6)
        plt.savefig(f"{compound}.svg", format='svg', orientation="landscape", quality=95, dpi = 100, bbox_inches="tight")
        plt.show()

    else:
        fig, ax = plt.subplots()
        ax.bar(pos, heights, yerr=yerrs, tick_label=labels, width=.3, linewidth=2, color=colors, capsize=5, error_kw={'lw':2}, edgecolor='black')
        ax.set_ylim(0, 1.5*max(aa_reference, cg_reference))
        ax_ticks = np.arange(0, max(ax.get_yticks()), 2)
        ax.set_yticks(ax_ticks)
        ax.axes.xaxis.set_ticklabels(labels, fontsize=ft_labels, weight='bold')
        ax.set_yticklabels(ax_ticks, fontsize=ft_labels, weight='bold')
        ax.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:.0f}'))
        ax.xaxis.tick_bottom()
        ax.tick_params(labeltop=False)  # don't put tick labels at the top
        ax.text(1, aa_reference+aa_reference_std+1, f'{aa_reference:.3f}', ha='center', fontsize=ft_labels, color='#595959', weight='bold')
        ax.text(1.5, cg_reference+cg_reference_std+1, f'{cg_reference:.3f}', ha='center', fontsize=ft_labels, color='#595959', weight='bold')

        d = .3  # proportion of vertical to horizontal extent of the slanted line

        for axis in ['top','bottom','left','right']:
            ax.spines[axis].set_linewidth(2)
        ax.tick_params(width=2, length=4)


        if compound == "DMNFQH2":
            compound = "DMNFQH$_2$"
        if compound == "UBQH2":
            compound = "UBQH$_2$"
        if compound == "UBQ0":
            compound = "UBQ-0"
        if compound == "NAD+":
            compound = "NAD$^+$"
        if compound == "NADP+":
            compound = "NADP$^+$"
        if compound == "UBQ0H2":
            compound = "UBQ0H$_2$"
        if compound == "XQH2":
            compound = "XQH$_2$"
        ax.set_title(f"{compound}", verticalalignment='center', fontsize=22, weight='bold', pad= "10.0")
        ax.set_ylabel("SASA (nm$^{2}$)", fontsize=ft_labels, weight='bold')
        figure = plt.gcf() # get current figure
        figure.set_size_inches(8, 6)
        plt.savefig(f"{compound}.svg", format='svg', orientation="landscape", quality=95, dpi = 100, bbox_inches="tight")
        plt.show()

In [ ]:
#ATP, NADPH, FAD, TPP - plot
pos = np.arange(1,2, 0.25)

ft_labels = 14

height_compound=[]
height_std=[]
uncertain = []
compounds = ["ADOS", "ADP", "ALO", "AMP", "ATP", "DMNFQ", "DMNFQH2", "FAD", "FMN", "GBZ", "HAQO", "HAQOH", "HQNO", "HQNOH", "ISO", "MND", "MNDOL", "MNQ", "MNQOL", "NAD+", "NADH", "NADP+", "NADPH", "NRBOS", "PQ", "PQOL", "RBFL", "RIB", "THI", "TMP", "TPP", "UBQ0", "UBQ0H2", "UBQN", "UBQNOL", "XQ", "XQH2"]

for compound in compounds:
    if compound == "ATP" or compound == "FAD" or compound == "NADPH" or compound == "TPP":          
        aa_reference = MeansAA[compound]
        aa_reference_ci = SASAsAACIS[compound]
        cg_reference = MeansCG[compound]
        cg_reference_ci = SASAsCGCIS[compound]
        aa = ufloat(aa_reference, np.abs(aa_reference_ci - aa_reference).mean())
        cg = ufloat(cg_reference, np.abs(cg_reference_ci - cg_reference).mean())
        values_uncertain = (cg/aa-1)*100
        uncertain.append(values_uncertain)
        print(compound, aa, cg)
    
compounds=["ATP", "FAD", "NADPH","TPP"]

   
fig, ax = plt.subplots()
ax.axhspan(95, 105, color="#32a85020", linestyle="--", linewidth=3)
ax.bar(pos, [u.nominal_value for u in uncertain], yerr=[u.std_dev for u in uncertain], bottom = 100, tick_label=compounds, width=0.15, linewidth=2, color=["#f39517", "#4bacc6", "#CF2D2A", "#7f7f7f"], capsize=5, error_kw={'lw':2}, edgecolor='black')
#ax.bar(pos, height_compound, yerr=height_std, tick_label=compounds, width=.3, linewidth=2, color=colors, capsize=5, error_kw={'lw':2}, edgecolor='black')
ax.axhline(y = 100, color="black")
ax.set_ylim(90, 110)
ax_ticks = np.arange(90, 112, 5)
ax.set_yticks(ax_ticks)
ax.axes.xaxis.set_ticklabels(compounds, fontsize=ft_labels, weight='bold')

ax.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:.0f}'))
ax.xaxis.tick_bottom()
ax.tick_params(labeltop=False)  # don't put tick labels at the top
ax.yaxis.set_major_formatter(mtick.PercentFormatter())

for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(2)
ax.tick_params(width=2, length=4)

#ax.set_title("CG-to-AA SASA ratio for ATP, NADPH, FAD and TPP", verticalalignment='center', fontsize=18, weight='bold', pad= "30.0")
ax.set_ylabel("CG-to-AA SASA ratio (%)", fontsize=ft_labels, weight='bold', labelpad=20)
#ax.set_xlabel("Compound", fontsize=ft_labels, weight='bold', labelpad=20)
ax.set_yticklabels(ax_ticks, fontsize=ft_labels, weight='bold')
figure = plt.gcf() # get current figure
figure.set_size_inches(5, 6)
plt.savefig("ratios_article.svg", format='svg', orientation="landscape", quality=95, dpi = 100, bbox_inches="tight")
plt.show()


In [ ]:
#All values 
uncertain = []
compounds = ["ADOS", "ADP", "ALO", "AMP", "ATP", "DMNFQ", "DMNFQH2", "FAD", "FMN", "GBZ", "HAQO", "HAQOH", "HQNO", "HQNOH", "ISO", "MND", "MNDOL", "MNQ", "MNQOL", "NAD+", "NADH", "NADP+", "NADPH", "NRBOS", "PQ", "PQOL", "RBFL", "RIB", "THI", "TMP", "TPP", "UBQ0", "UBQ0H2", "UBQN", "UBQNOL", "XQ", "XQH2"]

for compound in compounds:   
    aa_reference = MeansAA[compound]
    aa_reference_ci = SASAsAACIS[compound]
    cg_reference = MeansCG[compound]
    cg_reference_ci = SASAsCGCIS[compound]
    if aa_reference_ci != "None" or cg_reference_ci != "None":
        aa = ufloat(aa_reference, np.abs(aa_reference_ci - aa_reference).mean())
        cg = ufloat(cg_reference, np.abs(cg_reference_ci - cg_reference).mean())
        values_uncertain = (cg/aa-1)*100
        uncertain.append(values_uncertain)
    else:
        aa = aa_reference
        cg = cg_reference
        values_uncertain = (cg/aa-1)*100
        uncertain.append(values_uncertain)
    print(compound, "|||", values_uncertain, "|||", aa, "|||", cg)